In [2]:
!pip install -q langchain
!pip install -q openai


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import sqlite3
from pathlib import Path

parquet_files = Path("data/parquet").glob("*.parq")

dataframes = []
prefixes = []


for file_path in parquet_files:
    prefix_parts = file_path.stem.split("_")[:-1]
    prefix = "_".join(prefix_parts) if prefix_parts else file_path.stem

    df = pd.read_parquet(file_path)

    prefixes.append(prefix)
    dataframes.append(df)

    print(f"Loaded '{prefix}' dataframe with {len(df)} rows and {len(df.columns)} columns")

dataframes_dict = dict(zip(prefixes, dataframes))

print(f"\nLoaded {len(dataframes)} dataframes with prefixes: {prefixes}")

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Loaded 'Appointment' dataframe with 44309 rows and 39 columns
Loaded 'Company' dataframe with 36975 rows and 31 columns
Loaded 'Contact' dataframe with 48145 rows and 16 columns
Loaded 'cont_comp' dataframe with 51481 rows and 20 columns
Loaded 'Currency' dataframe with 160 rows and 12 columns
Loaded 'Interaction' dataframe with 321359 rows and 34 columns
Loaded 'Intr_Intr' dataframe with 44512 rows and 8 columns
Loaded 'Intr_Oppo' dataframe with 321359 rows and 7 columns
Loaded 'Intr_User' dataframe with 463364 rows and 12 columns
Loaded 'opportunity' dataframe with 133696 rows and 43 columns
Loaded 'Oppo_Comp' dataframe with 123124 rows and 8 columns
Loaded 'Oppo_Cont' dataframe with 116305 rows and 8 columns
Loaded 'Reference' dataframe with 19641 rows and 28 columns
Loaded 'User' dataframe with 4403 rows and 23 columns

Loaded 14 dataframes with prefixes: ['Appointment', 'Company', 'Contact', 'cont_comp', 'Currency', 'Interaction', 'Intr_Intr', 'Intr_Oppo', 'Intr_User', 'opportunit

In [4]:
len(dataframes)

14

In [5]:
OPENAI_API_KEY[:10]

'sk-proj-B9'

In [6]:
df.columns

Index(['userKey', 'userUserCode', 'userFullname', 'userSysCreatedDate',
       'userSysUpdatedDate', 'userKind', 'userIsActive', 'userCategory',
       'userAuthMethod', 'userUprivileges', 'userEmail', 'userRegion',
       'userRoleKey', 'userContactKey', 'userPhone', 'userLanguage',
       'userIsDepartment', 'userIsContact', 'userTimeZone', 'userSkill',
       'userTeamKind', 'userSysReplicationDate', 'userSysReplicationUserKey'],
      dtype='object')

In [27]:
conn = sqlite3.connect('Rag.sqlite')
c = conn.cursor()

In [30]:
c.execute('''CREATE TABLE IF NOT EXISTS Opportunity(
    oppoKey TEXT PRIMARY KEY,
    oppoName TEXT,
    oppoSysCreatedDate TEXT,
    oppoSysUpdatedDate TEXT,
    oppoSysCreatedUserKey TEXT,
    oppoUniqueRefIsFixed INTEGER,
    oppoTargetDate TEXT,
    oppoStatus TEXT,
    oppoNature TEXT,
    oppoSuccessProbability REAL,
    oppoBudget REAL,
    oppoMargin REAL,
    oppoDiscussionDate TEXT,
    oppoNegotiationDate TEXT,
    oppoConclusionDate TEXT,
    oppoCompetitors TEXT,
    oppoLostAgainst TEXT,
    oppoLostAgainstReason TEXT,
    oppoMemo TEXT,
    oppoMemoFormatted TEXT,
    oppoSysUsedDate TEXT,
    oppoBudgetConsolidated REAL,
    oppoDiscountAmount REAL,
    oppoBanner TEXT,
    oppoInvoiceSettings TEXT,
    oppoTags TEXT,
    oppoStatusChangeDate TEXT,
    oppoOriginKey TEXT,
    oppoSimpleStatus TEXT,
    oppoInvestmentAmount REAL,
    oppoCompetitorsMemo TEXT,
    oppoPartitionKey TEXT,
    oppoClosingReason TEXT,
    oppoCurrencyKey TEXT,
    oppoConsolidated INTEGER,
    oppoConsolidatedDate TEXT,
    oppoType TEXT,
    oppoSysReplicationDate TEXT,
    oppoSysReplicationUserKey TEXT,
    oppoArchived INTEGER,
    oppoCommitment TEXT,
    oppoRole TEXT,
    oppoQuoteSyncEnabled INTEGER
)''')
conn.commit()

In [31]:
def create_table_for_dataframe(df, table_name, conn, c):

    dtype_mapping = {
        'int64': 'INTEGER',
        'float64': 'REAL',
        'bool': 'INTEGER',
        'datetime64[ns]': 'TEXT',
        'object': 'TEXT',
    }

    columns = []

    for column, dtype in df.dtypes.items():
        sqlite_type = dtype_mapping.get(str(dtype), 'TEXT')
        if column == df.columns[0]:
            columns.append(f"{column} {sqlite_type} PRIMARY KEY")
        else: 
            columns.append(f"{column} {sqlite_type}")
        
    create_statement = f"CREATE TABLE IF NOT EXISTS {table_name} (\n    " + ",\n    ".join(columns) + "\n)"
    print(f"Creating table: '{table_name}'...")
    c.execute(create_statement)
    conn.commit()


def insert_data(df, table_name, conn):
    placeholders = ' , '.join(['?'] * len(df.columns))

    batch_size = 1000
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]
        c.executemany(f"INSERT OR REPLACE INTO {table_name} VALUES ({placeholders})", batch.values.tolist())
        conn.commit()
    print(f"Inserted {len(df)} rows into table: '{table_name}'...")



for i, df in enumerate(dataframes):
    table_name = f"{prefixes[i]}"
    create_table_for_dataframe(df, table_name, conn, c)
    insert_data(df, table_name, conn)

print("All tables created and data inserted successfully!")

Creating table: 'Appointment'...
Inserted 44309 rows into table: 'Appointment'...
Creating table: 'Company'...
Inserted 36975 rows into table: 'Company'...
Creating table: 'Contact'...
Inserted 48145 rows into table: 'Contact'...
Creating table: 'cont_comp'...
Inserted 51481 rows into table: 'cont_comp'...
Creating table: 'Currency'...
Inserted 160 rows into table: 'Currency'...
Creating table: 'Interaction'...
Inserted 321359 rows into table: 'Interaction'...
Creating table: 'Intr_Intr'...
Inserted 44512 rows into table: 'Intr_Intr'...
Creating table: 'Intr_Oppo'...
Inserted 321359 rows into table: 'Intr_Oppo'...
Creating table: 'Intr_User'...
Inserted 463364 rows into table: 'Intr_User'...
Creating table: 'opportunity'...
Inserted 133696 rows into table: 'opportunity'...
Creating table: 'Oppo_Comp'...
Inserted 123124 rows into table: 'Oppo_Comp'...
Creating table: 'Oppo_Cont'...
Inserted 116305 rows into table: 'Oppo_Cont'...
Creating table: 'Reference'...
Inserted 19641 rows into ta

In [32]:
for i, df in enumerate(dataframes): # Creates a table for each dataframe
    # Clean table name to ensure it's valid SQL
    table_name = ''.join(c if c.isalnum() else '_' for c in prefixes[i])
    
    try:
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        print(f"Successfully created table: {table_name}")
    except Exception as e:
        print(f"Error creating table {table_name}: {str(e)}")

Successfully created table: Appointment
Successfully created table: Company
Successfully created table: Contact
Successfully created table: cont_comp
Successfully created table: Currency
Successfully created table: Interaction
Successfully created table: Intr_Intr
Successfully created table: Intr_Oppo
Successfully created table: Intr_User
Error creating table opportunity: table "opportunity" already exists
Successfully created table: Oppo_Comp
Successfully created table: Oppo_Cont
Successfully created table: Reference
Successfully created table: User


In [33]:
c.execute('''SELECT * FROM Oppo_cont LIMIT 5''')
for row in c.fetchall():
    print(row)

('00010qZh00010Pdt', '00010qY40000wK56', '00010QH20000D1UL', '2004-03-29 15:04:41.000', 1, None, '2019-02-15 14:59:03.000', 0)
('00010qZh00010Pdu', '00010qY40000wK57', '00010QH20000D1DU', '2023-12-08 15:48:25.727', 1, '00010EZE00000ArA', '2019-02-15 14:58:47.000', 0)
('00010qZh00010Pdv', '00010qY40000wK57', '00010QH20000D2bG', '2023-12-08 15:48:25.400', 0, '00010EZE00000ArC', '2019-02-15 14:58:47.000', 0)
('00010qZh00010Pdw', '00010qY40000wK5A', '00010QH20000D5JG', '2023-12-08 15:48:25.670', 1, '00010EZE00000Ar6', '2019-02-15 14:58:48.000', 0)
('00010qZh00010Pdx', '00010qY40000wK5B', '00010QH20000D5JO', '2004-04-01 07:19:47.000', 1, '00010EZE00000kcN', '2019-02-15 14:58:48.000', 0)


In [34]:
def read_sql_query(sql, db):
    conn = sqlite3.connect(db)
    cur = conn.cursor()
    cur.execute(sql)
    rows = cur.fetchall()
    conn.close()
    return rows

db_file = 'Rag.sqlite'
sql_query = 'SELECT * FROM Opportunity LIMIT 5'
result_rows = read_sql_query(sql_query, db_file)

for row in result_rows:
    print(row)

('00010qY40000wK55', 'Aandarta - 10 User (Real Estate)', '2016-01-04 10:25:23.000', '2019-02-26 07:33:31.000', '000102bI00000nce', 0, '2016-01-04 10:02:03.000', '00010EZE000XmyIB', '00010EZE00000kYL', 0.0, 0.0, None, None, None, '2016-01-04 10:02:03.000', None, None, None, '0x1F8B08000000000000FFB551CB6EC230109C730E7C833FA0443C4A811C11AA5A559510178E1510870423476A42A5F2BDFD8C1E188F73A8D40327B4CA66763D3BBB5E6F50E38402076CE1E90DD6B0440D1CBEF1C0B8674618D0A7FC06987528E486981005C690B8673231E61D9AE011095EA955B087418E333EB14749FC82947EC14E39AD62DF929C8266C931E88B1FF085BE42CBD89311AB9D725EA8E6DF9167C9B19CDA60C5A8C69191535DD029C4AFA4D8EA561E3F526B54177ABE49ABE51E625DECF0C4EF5D275B3143EE4B1571861DF34133E55D3FEE6AC91FDBDC7CB5A566F69AB4D1AD0ECA4FB5F967E6773C396B7A83F89A63A131324623F9A96A927F3BBDE82D7FB941475DDB6DF1446B710561FC1F5252020000', '0', '2019-02-26 08:33:31.000', None, None, None, None, None, None, None, None, None, None, None, None, None, 0, None, None, '2019-02-26 07:33:31.000', '0', 0, 

In [16]:
pip install langchain langchain-openai langchain-community

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
pip install langchain-experimental

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
pip install -U langchain-openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
from langchain_openai import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [45]:
input_db = SQLDatabase.from_uri("sqlite:///Rag.sqlite")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0)
db_agent = SQLDatabaseChain.from_llm(llm, input_db, verbose=True)

In [46]:
from langchain.chains import create_sql_query_chain
from langchain_core.output_parsers import StrOutputParser

query_chain = create_sql_query_chain(llm, input_db)
sql_query = query_chain.invoke({"question": "Give me top 5 opportunities with the highest budget"})

result = input_db.run(sql_query)
print(result)


OperationalError: (sqlite3.OperationalError) near "```sql
SELECT "oppoKey", "oppoName", "oppoBudget" 
FROM "Opportunity" 
ORDER BY "oppoBudget" DESC 
LIMIT 5;
```": syntax error
[SQL: ```sql
SELECT "oppoKey", "oppoName", "oppoBudget" 
FROM "Opportunity" 
ORDER BY "oppoBudget" DESC 
LIMIT 5;
```]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [47]:
db_agent.run("Give me top 5 opportunities with the highest budget. Return the oppoKey, oppoName, and oppoBudget columns.")



> Entering new SQLDatabaseChain chain...
Give me top 5 opportunities with the highest budget. Return the oppoKey, oppoName, and oppoBudget columns.
SQLQuery:```sql
SELECT "oppoKey", "oppoName", "oppoBudget" 
FROM "Opportunity" 
ORDER BY "oppoBudget" DESC 
LIMIT 5;
```

OperationalError: (sqlite3.OperationalError) near "```sql
SELECT "oppoKey", "oppoName", "oppoBudget" 
FROM "Opportunity" 
ORDER BY "oppoBudget" DESC 
LIMIT 5;
```": syntax error
[SQL: ```sql
SELECT "oppoKey", "oppoName", "oppoBudget" 
FROM "Opportunity" 
ORDER BY "oppoBudget" DESC 
LIMIT 5;
```]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [24]:
db_agent.run("List the tables in the database.")



> Entering new SQLDatabaseChain chain...
List the tables in the database.
SQLQuery:SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
SQLResult: 
Answer:"artists", "albums", "media_types", "genres", "playlists", "playlist_track", "tracks"
> Finished chain.


'"artists", "albums", "media_types", "genres", "playlists", "playlist_track", "tracks"'

In [ ]:
db_agent.run("How the tables are related to each other ?")



> Entering new SQLDatabaseChain chain...
How the tables are related to each other ?
SQLQuery:SELECT * FROM sqlite_master WHERE type='table'
SQLResult: [('table', 'Opportunity', 'Opportunity', 2, 'CREATE TABLE "Opportunity" (\n"oppoKey" TEXT,\n  "oppoName" TEXT,\n  "oppoSysCreatedDate" TEXT,\n  "oppoSysUpdatedDate" TEXT,\n  "oppoSysCreatedUserKey" TEXT,\n  "oppoUniqueRefIsFixed" INTEGER,\n  "oppoTargetDate" TEXT,\n  "oppoStatus" TEXT,\n  "oppoNature" TEXT,\n  "oppoSuccessProbability" REAL,\n ...')]
Answer:The tables are related to each other through the "oppoKey" column, which is the primary key for the Opportunity table. This column is used to link data from other tables to the Opportunity table.
> Finished chain.


'The tables are related to each other through the "oppoKey" column, which is the primary key for the Opportunity table. This column is used to link data from other tables to the Opportunity table.'

In [ ]:
db_agent.run("List all the tables in the database.")



> Entering new SQLDatabaseChain chain...
List all the tables in the database.
SQLQuery:SELECT name FROM sqlite_master WHERE type='table'
SQLResult: [('Opportunity',)]
Answer:Opportunity
> Finished chain.


'Opportunity'